# Imports

In [1]:
import re, unicodedata, numpy as np, pandas as pd, pickle, os, math

from numpy import random
from google.colab import files

from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Download from kaggle

In [ ]:
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

Saving kaggle.json to kaggle.json


In [ ]:
! chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d mswarbrickjones/reddit-selfposts

 98% 345M/352M [00:04<00:00, 97.4MB/s]
100% 352M/352M [00:04<00:00, 86.4MB/s]


In [ ]:
! unzip reddit-selfposts.zip -d data

Archive:  reddit-selfposts.zip
  inflating: data/rspct.tsv          
  inflating: data/subreddit_info.csv  


# Data Processing Functions

In [2]:
def preprocess_text(text):
    text = text.lower()
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    text = re.sub(r'[^A-Za-z\'\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text

In [ ]:
def get_data(path):
  print('Getting data')
  raw_df = pd.read_table(path+'rspct.tsv')
  sub_df = pd.read_csv(path+'subreddit_info.csv')

  print('data read in. starting dataframe join:')

  df = pd.merge(raw_df, sub_df, how='inner',on='subreddit')\
            [['category_1','selftext']]\

  print('dataframes joined. starting text preprocessing:')

  df.selftext = df.selftext.apply(preprocess_text)

  df.category_1 = df.category_1.apply(lambda x: x.replace('/','&').replace(' ','').replace(' lb',''))

  print('Text processed.')
  return df


In [7]:
cats = ['writing&stories',
'tv_show',
'autos',
'hardware&tools',
'electronics']
# 'video_game',
# 'crypto',
# 'sports',
# 'hobby',
# 'appearance'
# ,
# 'card_game',
# 'drugs',
# 'advice&question',
# 'social_group',
# 'anime&manga',
# 'sex&relationships',
# 'software',
# 'health',
# 'animals',
# 'arts',
# 'programming',
# 'rpg',
# 'books',
# 'parenting',
# 'education',
# 'company&website',
# 'profession',
# 'music',
# 'politics&viewpoint',
# 'stem',
# 'travel',
# 'geo',
# 'religion&supernatural',
# 'board_game',
# 'movies',
# 'food&drink',
# 'finance&money',
# 'meta']

# Set paths

In [3]:
raw_data_path = 'data/'
drive_path = '/content/gdrive/MyDrive/CIS 522 Final Project/'

# Writing data to directory

In [ ]:
def prep_data(df, drive_path):
  print('Saving processed data:')

  num_cats = len(cats)

  for i, cat in enumerate(cats):
    print('Category {}/{}:{}'.format(i,num_cats,cat))

    data = df[df.category_1 == cat].selftext

    data.to_csv(drive_path + 'data/' + cat + '.csv',index=False,header=True)
  print('Done')

In [ ]:
df = get_data(raw_data_path)
prep_data(df, drive_path)

Getting data
data read in. starting dataframe join:
dataframes joined. starting text preprocessing:
Text processed.
Saving processed data:
Category 0/5:writing&stories
Category 1/5:tv_show
Category 2/5:autos
Category 3/5:hardware&tools
Category 4/5:electronics
Done


# Defining training

In [4]:
#Downloading proper software
!pip install transformers
!pip install datasets

     |████████████████████████████████| 2.1MB 19.4MB/s 
     |████████████████████████████████| 901kB 52.3MB/s 
     |████████████████████████████████| 3.3MB 49.0MB/s 
     |████████████████████████████████| 225kB 17.7MB/s 
     |████████████████████████████████| 245kB 48.9MB/s 
     |████████████████████████████████| 112kB 48.9MB/s 


In [5]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling,AutoTokenizer, AutoModelForCausalLM, pipeline
from datasets import Dataset

def train_models(cats, drive_path):
  print('Training models:')
 
  num_cats = len(cats)

  for i, cat in enumerate(cats, 1):
    print('Category {}/{}:{}'.format(i,num_cats,cat))
    tokenizer = AutoTokenizer.from_pretrained('distilgpt2')
    tokenizer.pad_token = tokenizer.eos_token
    model = AutoModelForCausalLM.from_pretrained('distilgpt2').to('cuda')
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


    data = Dataset.from_csv(drive_path + 'data/'+ cat + '.csv', split='train')

    def encode(batch): 
      return tokenizer([x[:500] for x in batch['selftext']], truncation=True, padding=True)

    processed = data.map(encode, batched=True, with_indices= False, batch_size=len(data))
    processed.set_format('torch', columns=['input_ids', 'attention_mask'])

    training_args = TrainingArguments(
        output_dir='./content/',
        overwrite_output_dir=True,
        num_train_epochs=3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=32,
        logging_steps=100,
        weight_decay=0.01,
        logging_dir='./logs/'
    )

    trainer = Trainer(
        model=model,
        tokenizer=tokenizer,
        args=training_args,
        data_collator=data_collator,
        train_dataset=processed,
    )

    trainer.train()
    trainer.save_model(drive_path + 'GPT_generators/' + cat + '/')

# Training models

In [ ]:
train_models([cats[0]],drive_path)

Training models:
Category 1/1:writing&stories


Using custom data configuration default-447a5a1dae6b9d9d
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-447a5a1dae6b9d9d/0.0.0)
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-447a5a1dae6b9d9d/0.0.0/cache-e479a7c8abf6731f.arrow


Step,Training Loss
100,4.588300
200,4.442600
300,4.368300
400,4.316500
500,4.306200
600,4.291100
700,4.287500
800,4.236100
900,4.218900
1000,4.242500


In [ ]:
train_models(cats,drive_path)

Training models:
Category 1/5:writing&stories


Using custom data configuration default-447a5a1dae6b9d9d


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-447a5a1dae6b9d9d/0.0.0. Subsequent calls will reuse this data.


Step,Training Loss
100,4.589600
200,4.445400
300,4.373700
400,4.324500
500,4.316800
600,4.304400
700,4.306800
800,4.259200
900,4.250800
1000,4.281800


Category 2/5:tv_show


...

Step,Training Loss
100,4.605800
200,4.419700
300,4.342100
400,4.267100
500,4.283100
600,4.231600
700,4.221800
800,4.230900
900,4.187600
1000,4.126100


Category 5/5:electronics


Using custom data configuration default-d437147d7628a3ee


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-d437147d7628a3ee/0.0.0. Subsequent calls will reuse this data.


Step,Training Loss
100,4.578800
200,4.402700
300,4.331200
400,4.271800
500,4.283800
600,4.232700
700,4.225700
800,4.214400
900,4.207700
1000,4.186700


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


# Text generation

In [ ]:
for cat in cats[:10]:
  print('Category:',cat)
  mdl = pipeline('text-generation', 
                  model=drive_path + 'GPT_generators/' + cat + '/', 
                  device=0)
  print('Text:',mdl('What I think ')[0]['generated_text'],'\n')

Category: writing&stories


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Text: What I think ive read so far is a fantastic description of the project i'm really looking forward to doing lb lb i'm looking for a writer to fill the role of the girl lb lb lb if you have any other ideas in mind please drop 

Category: tv_show


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Text: What I think ive heard has been a relatively new phenomenon i've been following for awhile and i feel like the rise in frequency of stories on the web and in my experience a lot of people want to share in the world lb lb for example the 

Category: autos


...

Text: What I think ive been driving a lot ever since the 's was an excellent toyota for years and a few times i had a hard time getting past the clutches and it seemed obvious that the gs needed for larger vehicles like the i 

Category: hardware&tools


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Text: What I think ive just made a mistake i'm planning on replacing the scooter i've owned since i was a kid but im looking for something that's both fun and fun and i need to carry a scooter i've never had a sc 

Category: electronics


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Text: What I think ive tried and tried the zx gopro and the xy and ive tried them as a side note all my favorite features were the motion detection for the xy and the motion detection for the xy and xy 



#Perplexity Calculation

In [83]:
import torch
import numpy as np
text = "I just finished Scandal and I'm looking for a new show."
tokenizer = AutoTokenizer.from_pretrained('distilgpt2')
tokenizer.pad_token = tokenizer.eos_token
    
for cat in cats:
  print("Category:",cat)
  model = AutoModelForCausalLM.from_pretrained(drive_path + 'GPT_generators/' + cat + '/')
  encodings = tokenizer(text, return_tensors='pt')

  max_length = model.config.n_positions
  stride = 512

  lls = []
  for i in range(0, encodings.input_ids.size(1), stride):
      begin_loc = max(i + stride - max_length, 0)
      end_loc = min(i + stride, encodings.input_ids.size(1))
      trg_len = end_loc - i    # may be different from stride on last loop
      input_ids = encodings.input_ids[:,begin_loc:end_loc]
      target_ids = input_ids.clone()
      target_ids[:,:-trg_len] = -100

      with torch.no_grad():
          outputs = model(input_ids, labels=target_ids)
          log_likelihood = outputs[0] * trg_len

      lls.append(log_likelihood)

  ppl = torch.exp(torch.stack(lls).sum() / end_loc)
  print(ppl.item(),'\n')

Category: writing&stories
254.6935272216797 

Category: tv_show
51.117374420166016 

Category: autos
177.2567901611328 

Category: hardware&tools
122.44670104980469 

Category: electronics
115.01026153564453 

